In [1]:
import os
%pip install python-dotenv
%pip install -U langchain langchain_experimental langchain-openai


  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.3 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 kB 28.0 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 31.3 MB/s eta 

In [9]:
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_KEY")

In [10]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_experimental.tabular_synthetic_data.openai import (
    OPENAI_TEMPLATE,
    create_openai_data_generator,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import List, Dict

In [11]:
#create a schema for transactions
#transaction categories include housing, healthcare, transportation, utilities, travel, dining, groceries
class SpendingSchema(BaseModel):
    user_id: str       
    month: str        
    housing: int   
    bills_utilities: int
    dining: int
    transport: int
    fitness: int
    travel: int
    entertainment: int
    savings: int


In [12]:

examples = [
    {
        "example": """User ID: U001, Month: January 2023, Housing: 800, Bills & Utilities: 210, Groceries: 400, Dining: 400, Transport: 100, Fitness: 60, Travel: 400, Entertainment: 300, Savings: 1500"""
    },
    {
        "example": """User ID: U002, Month: January 2023, Housing: 1300, Bills & Utilities: 150, Groceries: 600, Dining: 200, Transport: 250, Fitness: 20, Travel: 100, Entertainment: 700, Savings: 600"""
    }
]

In [13]:

OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")
prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)


In [14]:
synthetic_data_generator = create_openai_data_generator(
    output_schema=SpendingSchema,
    llm=ChatOpenAI(
        temperature=1,
        api_key=api_key
    ), 
    prompt=prompt_template,
)

PydanticUserError: `SyntheticDataGenerator` is not fully defined; you should define `BaseCache`, then call `SyntheticDataGenerator.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.10/u/class-not-fully-defined

In [ ]:
synthetic_results = synthetic_data_generator.generate(
    subject="spending_data",
    extra="Generate samples with dollar values for the transactions given that represent what people would spend and save. Make sure the year and month is before November 2024 to make it realistic. You can also add outliers in certain categories to represent people who go extreme in some areas." +
    "You can make housing between 600 and 10000, bills and utilities between 50 and 1000, dining between 50 and 1000, transport between 20 and 500, fitness between 0 and 400, travel between 0 and 4000, entertainment between 10 and 3000, and savings between 0 and 3000. Make it as realistic as possible",
    runs=1000,
)

In [ ]:
len(synthetic_results)

10

In [ ]:
synthetic_results

[SpendingSchema(user_id='U003', month='August 2023', housing=5000, bills_utilities=800, dining=600, transport=400, fitness=200, travel=2000, entertainment=2500, savings=2500),
 SpendingSchema(user_id='U004', month='March 2024', housing=2500, bills_utilities=300, dining=800, transport=350, fitness=50, travel=1500, entertainment=1200, savings=1500),
 SpendingSchema(user_id='U005', month='June 2024', housing=7200, bills_utilities=600, dining=900, transport=300, fitness=250, travel=3500, entertainment=2800, savings=2700),
 SpendingSchema(user_id='U006', month='August 2023', housing=4000, bills_utilities=200, dining=500, transport=150, fitness=75, travel=3000, entertainment=2000, savings=2000),
 SpendingSchema(user_id='U007', month='July 2022', housing=7800, bills_utilities=450, dining=800, transport=200, fitness=150, travel=3700, entertainment=2500, savings=2800),
 SpendingSchema(user_id='U008', month='September 2023', housing=7500, bills_utilities=300, dining=700, transport=180, fitness=1

In [20]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_experimental.tabular_synthetic_data.openai import (
    create_openai_data_generator,
)
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Optional, List, Dict

# Define Spending Schema using Pydantic
class SpendingSchema(BaseModel):
    user_id: str = Field(..., description="Unique identifier for the user")
    month: str = Field(..., description="Month of the spending")
    housing: int = Field(..., description="Spending on housing")
    bills_utilities: int = Field(..., description="Spending on bills and utilities")
    dining: int = Field(..., description="Spending on dining")
    transport: int = Field(..., description="Spending on transportation")
    fitness: int = Field(..., description="Spending on fitness activities")
    travel: int = Field(..., description="Spending on travel")
    entertainment: int = Field(..., description="Spending on entertainment")
    savings: int = Field(..., description="Savings for the month")

# Examples for FewShotPromptTemplate
examples = [
    {
        "subject": "User ID: U001, Month: January 2023 spending",
        "extra": "Housing: 800, Bills & Utilities: 210, Dining: 400, Transport: 100, Fitness: 60, Travel: 400, Entertainment: 300, Savings: 1500"
    },
    {
        "subject": "User ID: U002, Month: February 2023 spending",
        "extra": "Housing: 1300, Bills & Utilities: 150, Dining: 200, Transport: 250, Fitness: 20, Travel: 100, Entertainment: 700, Savings: 600"
    }
]

# Define the example template
example_template = PromptTemplate(
    input_variables=["subject", "extra"],
    template="{subject}:\n{extra}"
)

# Define the few-shot prompt template
few_shot_prompt = FewShotPromptTemplate(
    prefix="Below are examples of spending data:",
    examples=examples,
    suffix="Generate realistic synthetic spending data with variations.",
    input_variables=["subject", "extra"],
    example_prompt=example_template
)

# Initialize ChatOpenAI
llm = ChatOpenAI(
    temperature=0.7,
    model="gpt-4",  # Ensure a supported model is used
    api_key="your_openai_api_key"  # Replace with your OpenAI API key
)

# Create synthetic data generator
synthetic_data_generator = create_openai_data_generator(
    output_schema=SpendingSchema,
    llm=llm,
    prompt=few_shot_prompt,
)

# Generate synthetic data
synthetic_results = synthetic_data_generator.generate(
    subject="User ID: U003, March 2023 spending",
    extra=(
        "Housing: 1000-5000, Bills & Utilities: 100-1000, Dining: 50-800, "
        "Transport: 50-400, Fitness: 0-300, Travel: 0-2000, Entertainment: "
        "50-1000, Savings: 100-3000. Include some outliers for variety."
    ),
    runs=10
)

# Print results
for idx, result in enumerate(synthetic_results, start=1):
    print(f"Sample {idx}: {result.dict()}")
    print("-" * 50)



/var/folders/3h/d79ycrgx64v30y71jr_0ljlm0000gn/T/ipykernel_76062/488733686.py:50: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


PydanticUserError: `SyntheticDataGenerator` is not fully defined; you should define `BaseCache`, then call `SyntheticDataGenerator.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.10/u/class-not-fully-defined

In [24]:
import random
import pandas as pd
from faker import Faker

# Initialize Faker for generating user profiles
faker = Faker()

# Define spending categories and realistic ranges for spending
categories = {
    "housing": (800, 5000),
    "bills_utilities": (50, 1000),
    "dining": (50, 800),
    "transport": (20, 400),
    "fitness": (0, 300),
    "travel": (0, 2000),
    "entertainment": (10, 1000),
    "savings": (100, 5000),
}

def generate_user_data(user_id, month, year):
    """Generates realistic spending data for a single user."""
    data = {
        "user_id": user_id,
        "month": month,
        "year": year,
    }
    for category, (min_val, max_val) in categories.items():
        # Add realistic variation and outliers
        if random.random() < 0.05:  # 5% chance of an extreme value (outlier)
            data[category] = random.randint(max_val, max_val * 2)
        else:
            data[category] = random.randint(min_val, max_val)
    return data

def generate_synthetic_dataset(num_users=100, months=12):
    """Generates a synthetic dataset for multiple users over a period."""
    dataset = []
    for _ in range(num_users):
        user_id = faker.unique.user_name()
        for month in range(1, months + 1):
            year = 2023
            month_name = pd.Timestamp(year=year, month=month, day=1).strftime("%B")
            dataset.append(generate_user_data(user_id, month_name, year))
    return pd.DataFrame(dataset)

# Generate a dataset for 100 users over 12 months
synthetic_dataset = generate_synthetic_dataset(num_users=10000, months=12)

# Save to CSV or view as a sample
synthetic_dataset.to_csv("synthetic_spending_data.csv", index=False)
print(synthetic_dataset.head())


          user_id     month  year  housing  bills_utilities  dining  \
0  hansonbrittany   January  2023     2922              671     224   
1  hansonbrittany  February  2023      844              848      96   
2  hansonbrittany     March  2023     2489             1251     189   
3  hansonbrittany     April  2023     1858              583     792   
4  hansonbrittany       May  2023     3132              822     131   

   transport  fitness  travel  entertainment  savings  
0        256      236    1437            913     1487  
1         60      111     570            922      232  
2        281      267    1898            543     3869  
3        280      112     349            581     5319  
4        275       97     385            776      894  
